In [6]:
print('hello class')

hello class


In [7]:
%pip install -qU langchain langchain-openai langchain_chroma langchain_community langchainhub beautifulsoup4 bs4 rich

Note: you may need to restart the kernel to use updated packages.


In [8]:
import os

from dotenv import load_dotenv

load_dotenv()  # take environment variables

token = os.getenv("OPENAI_API_KEY")
endpoint = "https://models.github.ai/inference"
model = "openai/gpt-4.1-nano"

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=model, base_url=endpoint, api_key=token)

In [10]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from rich import print

In [20]:
# Load, chunk and index the contents of the blog.
import bs4
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2017-06-21-overview/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
splits = text_splitter.split_documents(docs)

print(splits)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(
    model="text-embedding-3-small",
    base_url="https://models.inference.ai.azure.com",
    api_key=token,
))


[
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='An Overview of Deep Learning for Curious People\n    \nDate: June 21, 2017  |  Estimated 
Reading Time: 12 min  |  Author: Lilian Weng'
    ),
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='(The post was originated from my talk for WiMLDS x Fintech meetup hosted by Affirm.)\nI 
believe many of you have watched or heard of the games between AlphaGo and professional Go player Lee Sedol in 
2016. Lee has the highest rank of nine dan and many world championships. No doubt, he is one of the best Go players
in the world, but he lost by 1-4 in this series versus AlphaGo. Before this, Go was considered to be an intractable
game for computers to master, as its simple rules lay out an exponential number of variations in the board 
positions, many more than what in Chess. This event surely highlighted 2016 as a big year for AI. Because of 
AlphaGo, much attention has been attracted to the progress of AI.'
    ),
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='Meanwhile, many companies are spending resources on pushing the edges of AI applications, 
that indeed have the potential to change or even revolutionize how we are gonna live. Familiar examples include 
self-driving cars, chatbots, home assistant devices and many others. One of the secret receipts behind the progress
we have had in recent years is deep learning.\nWhy Does Deep Learning Work Now?#\nDeep learning models, in simple 
words, are large and deep artificial neural nets. A neural network (“NN”) can be well presented in a directed 
acyclic graph: the input layer takes in signal vectors; one or multiple hidden layers process the outputs of the 
previous layer. The initial concept of a neural network can be traced back to more than half a century ago. But why
does it work now? Why do people start talking about them all of a sudden?'
    ),
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='A three-layer artificial neural network. (Image source: 
http://cs231n.github.io/convolutional-networks/#conv)\n\nThe reason is surprisingly simple:\n\nWe have a lot more 
data.\nWe have much powerful computers.'
    ),
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='A large and deep neural network has many more layers + many more nodes in each layer, which 
results in exponentially many more parameters to tune. Without enough data, we cannot learn parameters efficiently.
Without powerful computers, learning would be too slow and insufficient.\nHere is an interesting plot presenting 
the relationship between the data scale and the model performance, proposed by Andrew Ng in his “Nuts and Bolts of 
Applying Deep Learning” talk. On a small dataset, traditional algorithms (Regression, Random Forests, SVM, GBM, 
etc.) or statistical learning does a great job, but once the data scale goes up to the sky, the large NN 
outperforms others. Partially because compared to a traditional ML model, a neural network model has many more 
parameters and has the capability to learn complicated nonlinear patterns. Thus we expect the model to pick the 
most helpful features by itself without too much expert-involved manual feature engineering.'
    ),
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='The data scale versus the model performance. (Recreated based on: 
https://youtu.be/F1ka6a13S9I)'
    ),
    Document(
        metadata={'source': 'https://lilianweng.github.io/posts/2017-06-21-overview/'},
        page_content='Deep Learning Models#\nNext, let’s go through a few classical deep learning 
models.\nConvoluti

In [15]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

print(prompt)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


ChatPromptTemplate(
    input_variables=['context', 'question'],
    input_types={},
    partial_variables={},
    metadata={
        'lc_hub_owner': 'rlm',
        'lc_hub_repo': 'rag-prompt',
        'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'
    },
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['context', 'question'],
                input_types={},
                partial_variables={},
                template="You are an assistant for question-answering tasks. Use the following pieces of retrieved 
context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences 
maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"
            ),
            additional_kwargs={}
        )
    ]
)

In [17]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [18]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [19]:
rag_chain.invoke("What is Convolutional Neural Networks?")


'Convolutional Neural Networks (CNNs) are a type of feed-forward artificial neural network inspired by the organization of the visual cortex. They use convolutional layers with kernels or filters that slide across input images to detect features like edges and patterns. CNNs are widely used for image recognition and processing tasks.'